In [1]:
!nvidia-smi

Sun Oct 30 21:17:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   29C    P8     9W / 350W |     17MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 필요 패키지 다운로드

## 모듈 import

In [8]:
import json
import os

import torch
import torch.nn as nn
from tqdm import trange
from transformers import XLMRobertaModel, AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from datasets import load_metric
from sklearn.metrics import f1_score
import pandas as pd
import copy

## 전역 변수 설정
구글 드라이브 마운트 기준으로 설정되어 있음

In [36]:
PADDING_TOKEN = 1
S_OPEN_TOKEN = 0
S_CLOSE_TOKEN = 2

do_eval=True

category_extraction_model_path = './saved_model/category_extraction_emb/' 
polarity_classification_model_path = './saved_model/polarity_classification_emb/'

test_category_extraction_model_path = './saved_model/category_extraction_emb/saved_model_epoch_15.pt'
test_polarity_classification_model_path = './saved_model/polarity_classification_emb/saved_model_epoch_15.pt'

train_data_path = './data/nikluge-sa-2022-train.jsonl'
dev_data_path = './data/nikluge-sa-2022-dev.jsonl'
test_data_path = './data/nikluge-sa-2022-test.jsonl'

max_len = 256
batch_size = 8
base_model = 'monologg/koelectra-base-v3-discriminator'
learning_rate = 3e-6
eps = 1e-8
num_train_epochs = 20
classifier_hidden_size = 768
emb_classifier_hidden_size = 1280 # 기존 768에서 -> 1024 임베딩을 concat한 vector
classifier_dropout_prob = 0.1
lstm_hidden = 256
lstm_num_layer = 1
hidden_dropout_prob = 0.3
bilstm_flag = True

entity_property_pair = [
    '제품 전체#일반', '제품 전체#가격', '제품 전체#디자인', '제품 전체#품질', '제품 전체#편의성', '제품 전체#인지도', '제품 전체#다양성',
    '본품#일반', '본품#디자인', '본품#품질', '본품#편의성', '본품#다양성', '본품#가격', '본품#인지도',
    '패키지/구성품#일반', '패키지/구성품#디자인', '패키지/구성품#품질', '패키지/구성품#편의성', '패키지/구성품#다양성', '패키지/구성품#가격',
    '브랜드#일반', '브랜드#가격', '브랜드#디자인', '브랜드#품질', '브랜드#인지도' ]

entity_property_to_id = {entity_property_pair[i] : i for i in range(len(entity_property_pair))}
print(entity_property_to_id)

tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

special_tokens_dict = {
    'additional_special_tokens': ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
}

entity_property_seq = [ i for i in range(len(entity_property_pair))]

{'제품 전체#일반': 0, '제품 전체#가격': 1, '제품 전체#디자인': 2, '제품 전체#품질': 3, '제품 전체#편의성': 4, '제품 전체#인지도': 5, '제품 전체#다양성': 6, '본품#일반': 7, '본품#디자인': 8, '본품#품질': 9, '본품#편의성': 10, '본품#다양성': 11, '본품#가격': 12, '본품#인지도': 13, '패키지/구성품#일반': 14, '패키지/구성품#디자인': 15, '패키지/구성품#품질': 16, '패키지/구성품#편의성': 17, '패키지/구성품#다양성': 18, '패키지/구성품#가격': 19, '브랜드#일반': 20, '브랜드#가격': 21, '브랜드#디자인': 22, '브랜드#품질': 23, '브랜드#인지도': 24}


## Json 파일 읽어오는 함수

In [10]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j


# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

jsonlload('./data/sample.jsonl')

[{'id': 'nikluge-sa-2022-train-00001',
  'sentence_form': '둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악.',
  'annotation': [['본품#품질', ['기어', 16, 18], 'negative']]},
 {'id': 'nikluge-sa-2022-train-00002',
  'sentence_form': '이거 뭐 삐꾸를 준 거 아냐 불안하고, 거금 투자한 게 왜 이래.. 싶어서 정이 확 떨어졌는데 산 곳 가져가서 확인하니 기어 텐션 문제라고 고장 아니래.',
  'annotation': [['본품#품질', ['기어 텐션', 67, 72], 'negative']]},
 {'id': 'nikluge-sa-2022-train-00003',
  'sentence_form': '간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.',
  'annotation': [['제품 전체#일반', [None, 0, 0], 'positive']]},
 {'id': 'nikluge-sa-2022-train-00004',
  'sentence_form': '샥이 없는 모델이라 일반 도로에서 타면 노면의 진동 때문에 손목이 덜덜덜 떨리고 이가 부딪칠 지경인데 이마저도 며칠 타면서 익숙해지니 신경쓰이지 않게 됐다.',
  'annotation': [['제품 전체#일반', ['샥이 없는 모델', 0, 8], 'neutral']]},
 {'id': 'nikluge-sa-2022-train-00005',
  'sentence_form': '안장도 딱딱해서 엉덩이가 아팠는데 무시하고 타고 있다.',
  'annotation': [['본품#일반', ['안장', 0, 2], 'negative']]},
 {'id': 'nikluge-sa-2022-train-00006',
  'sentence_form': '지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는..',
  'annotation'

## 모델 정의
xlm-roberta 모델을 기반으로 한 classification 모델 이용

In [23]:
class SimpleClassifier(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        self.dense = nn.Linear(classifier_hidden_size, classifier_hidden_size)
        self.dropout = nn.Dropout(classifier_dropout_prob)
        self.output = nn.Linear(classifier_hidden_size, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

class EmbeddingClassifier(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        self.dense = nn.Linear(emb_classifier_hidden_size, classifier_hidden_size)
        self.dropout = nn.Dropout(classifier_dropout_prob)
        self.output = nn.Linear(classifier_hidden_size, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

class CategoryClassification(nn.Module):
    def __init__(self, num_label, len_tokenizer, category_emb_size, category_size, num_layer, bilstm_flag):
        super().__init__()

        assert category_emb_size == lstm_hidden * 2, "Please set category-embedding-size to twice the lstm-hidden-size"

        self.num_label = num_label # 0, 1 -> 0이면 해당 카테고리는 있는 거임.
        self.electra = ElectraModel.from_pretrained(base_model)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.n_hidden = lstm_hidden

        self.category_emb = nn.Embedding(category_size, category_emb_size, scale_grad_by_freq=True)

        self.num_layers = num_layer
        self.bidirectional = 2 if bilstm_flag else 1

        self.category_lstm_first = nn.LSTM(768, 256, bidirectional=True, batch_first=True)
        self.category_lstm_last = nn.LSTM(lstm_hidden * 4, self.n_hidden, num_layers=self.num_layers, batch_first=True, bidirectional=bilstm_flag)

        self.category_q_liner = nn.Linear(self.n_hidden * 2, self.n_hidden * 2)
        self.category_k_liner = nn.Linear(self.n_hidden * 2, self.n_hidden * 2)
        self.category_v_liner = nn.Linear(self.n_hidden * 2, self.n_hidden * 2)

        self.dropout = nn.Dropout(hidden_dropout_prob)

        self.softmax = nn.Softmax(dim=-1)

        self.labels_classifier = EmbeddingClassifier(self.num_label) # 입력 사이즈는 classifier_hidden_size = (768, 2) 0과 1 -> 


    def forward(self, input_ids=None, attention_mask=None, labels=None, category_label_seq_tensor=None, token_type_ids=None ):

        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        # (batch_size, max_length, hidden_size)
        sequence_output = outputs[0] # (batch_size, 256, 768)
        # print("sequence_output size:", sequence_output.shape)

        category_embs = self.category_emb(category_label_seq_tensor)
        # print(category_label_seq_tensor)
        # print('category_embs size:', category_embs.shape)

        hidden = None
        scaler = self.n_hidden ** 0.5

        """
        category predict layer
        """
        category_lstm_outputs, hidden = self.category_lstm_first(sequence_output, hidden)
        category_lstm_outputs = self.dropout(category_lstm_outputs)

        # print('category_lstm_outputs size:',category_lstm_outputs.shape)

        category_q = self.category_q_liner(category_lstm_outputs)
        category_k = self.category_k_liner(category_embs)
        category_v = self.category_v_liner(category_embs)

        # print("category_q", category_q.shape)
        # print("category_k", category_k.shape)
        # print("category_v", category_v.shape)

        category_attention_score = category_q.matmul(category_k.permute(0, 2, 1)) / scaler
        category_attention_align = self.softmax(category_attention_score)
        # print("category_attention_score", category_attention_score)

        category_attention_output = category_attention_align.matmul(category_v)
        category_attention_output = self.dropout(category_attention_output)
        # print("category_attention_output", category_attention_output.shape)


        category_lstm_outputs = torch.cat([category_lstm_outputs, category_attention_output], dim=-1)
        # print("category_lstm_outputs:", category_lstm_outputs.shape)

        category_lstm_outputs, hidden = self.category_lstm_last(category_lstm_outputs, hidden)
        category_lstm_outputs = self.dropout(category_lstm_outputs) # 요걸 다음층에 입력에 넣어야 하나..?

        # print("category_lstm_outputs", category_lstm_outputs.shape)

        category_q = self.category_q_liner(category_lstm_outputs)
        category_k = self.category_k_liner(category_embs)

        # print("category_q_last", category_q.shape)
        # print("category_k_last", category_k.shape)

        category_attention_score = category_q.matmul(category_k.permute(0, 2, 1)) / scaler
        category_attention_score = self.dropout(category_attention_score)

        final_category_attention_score = category_attention_score[:, 0, :]

        category_attention_score = category_attention_score.matmul(category_embs) # [batch_size, max_length, max_length]
        category_attention_score = torch.cat([sequence_output, category_attention_score], dim=-1)

        # print(category_attention_score.shape)
      
        logits = self.labels_classifier(category_attention_score)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

class RoBertaBaseClassifier(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(RoBertaBaseClassifier, self).__init__()

        self.num_label = num_label
        self.electra = ElectraModel.from_pretrained(base_model)
        self.electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        # print(sequence_output.shape)
        logits = self.labels_classifier(sequence_output)
        # print(logits.shape)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

## 데이터 파싱 및 토크나이저 정의

In [24]:
def tokenize_and_align_labels(tokenizer, form, annotations, max_len):

    entity_property_data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'label': [],
        'category': []
    }
    polarity_data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'label': []
    }

    for pair in entity_property_pair:
        isPairInOpinion = False
        if pd.isna(form):
            break
        tokenized_data = tokenizer(form, pair, padding='max_length', max_length=max_len, truncation=True)
        for annotation in annotations:
            entity_property = annotation[0]
            polarity = annotation[2]

            if polarity == '------------':
                continue

            category_seq = [ i for i in range(len(entity_property_pair))]    

            if entity_property == pair:
                entity_property_data_dict['input_ids'].append(tokenized_data['input_ids'])
                entity_property_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
                entity_property_data_dict['label'].append(tf_name_to_id['True'])
                entity_property_data_dict['category'].append(category_seq)
                # print(entity_property_data_dict)

                polarity_data_dict['input_ids'].append(tokenized_data['input_ids'])
                polarity_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
                polarity_data_dict['label'].append(polarity_name_to_id[polarity])

                isPairInOpinion = True
                break

        if isPairInOpinion is False:
            entity_property_data_dict['input_ids'].append(tokenized_data['input_ids'])
            entity_property_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
            entity_property_data_dict['label'].append(tf_name_to_id['False'])
            entity_property_data_dict['category'].append(category_seq) # 개체 없음(즉, 틀린 개체#속성 pair의 경우)

    return entity_property_data_dict, polarity_data_dict


def get_dataset(raw_data, tokenizer, max_len):
    input_ids_list = []
    attention_mask_list = []
    token_labels_list = []
    category_seq_list = []

    polarity_input_ids_list = []
    polarity_attention_mask_list = []
    polarity_token_labels_list = []

    for utterance in raw_data:
        entity_property_data_dict, polarity_data_dict = tokenize_and_align_labels(tokenizer, utterance['sentence_form'], utterance['annotation'], max_len)
        input_ids_list.extend(entity_property_data_dict['input_ids'])
        attention_mask_list.extend(entity_property_data_dict['attention_mask'])
        token_labels_list.extend(entity_property_data_dict['label'])
        category_seq_list.extend(entity_property_data_dict['category'])
       
        polarity_input_ids_list.extend(polarity_data_dict['input_ids'])
        polarity_attention_mask_list.extend(polarity_data_dict['attention_mask'])
        polarity_token_labels_list.extend(polarity_data_dict['label'])

    return TensorDataset(torch.tensor(input_ids_list), torch.tensor(attention_mask_list),
                         torch.tensor(token_labels_list), torch.tensor(category_seq_list)), TensorDataset(torch.tensor(polarity_input_ids_list), torch.tensor(polarity_attention_mask_list),
                         torch.tensor(polarity_token_labels_list))

## 모델 학습

In [25]:
def evaluation(y_true, y_pred, label_len):
    count_list = [0]*label_len
    hit_list = [0]*label_len
    for i in range(len(y_true)):
        count_list[y_true[i]] += 1
        if y_true[i] == y_pred[i]:
            hit_list[y_true[i]] += 1
    acc_list = []

    for i in range(label_len):
        acc_list.append(hit_list[i]/count_list[i])

    print(count_list)
    print(hit_list)
    print(acc_list)
    print('accuracy: ', (sum(hit_list) / sum(count_list)))
    print('macro_accuracy: ', sum(acc_list) / 3)
    # print(y_true)

    y_true = list(map(int, y_true))
    y_pred = list(map(int, y_pred))

    print('f1_score: ', f1_score(y_true, y_pred, average=None))
    print('f1_score_micro: ', f1_score(y_true, y_pred, average='micro'))
    print('f1_score_macro: ', f1_score(y_true, y_pred, average='macro'))

def train_sentiment_analysis():

    print('train_sentiment_analysis')
    print('category_extraction model would be saved at ', category_extraction_model_path)
    print('polarity model would be saved at ', polarity_classification_model_path)

    print('loading train data...')
    train_data = jsonlload(train_data_path)
    dev_data = jsonlload(dev_data_path)

    print('tokenizing train data...')
    tokenizer = ElectraTokenizer.from_pretrained(base_model)
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
    print('We have added', num_added_toks, 'tokens')

    print('making dataset...')
    entity_property_train_data, polarity_train_data = get_dataset(train_data, tokenizer, max_len)
    entity_property_dev_data, polarity_dev_data = get_dataset(dev_data, tokenizer, max_len)

    print('making dataloader...')
    entity_property_train_dataloader = DataLoader(entity_property_train_data, shuffle=True,
                                  batch_size=batch_size)
    entity_property_dev_dataloader = DataLoader(entity_property_dev_data, shuffle=True,
                                batch_size=batch_size)

    polarity_train_dataloader = DataLoader(polarity_train_data, shuffle=True,
                                                  batch_size=batch_size)
    polarity_dev_dataloader = DataLoader(polarity_dev_data, shuffle=True,
                                                batch_size=batch_size)

    print('loading model...')
    # entity_property_model = RoBertaBaseClassifier(len(tf_id_to_name), len(tokenizer))
    entity_property_model = CategoryClassification(len(tf_id_to_name), len(tokenizer), lstm_hidden * 2, len(entity_property_pair), lstm_num_layer, bilstm_flag)
    entity_property_model.to(device)

    polarity_model = RoBertaBaseClassifier(len(polarity_id_to_name), len(tokenizer))
    polarity_model.to(device)

    print('end loading')

    # entity_property_model_optimizer_setting
    FULL_FINETUNING = True
    if FULL_FINETUNING:
        entity_property_param_optimizer = list(entity_property_model.named_parameters())
        no_decay = ['bias', 'gamma', 'beta']
        entity_property_optimizer_grouped_parameters = [
            {'params': [p for n, p in entity_property_param_optimizer if not any(nd in n for nd in no_decay)],
             'weight_decay_rate': 0.01},
            {'params': [p for n, p in entity_property_param_optimizer if any(nd in n for nd in no_decay)],
             'weight_decay_rate': 0.0}
        ]
    else:
        entity_property_param_optimizer = list(entity_property_model.classifier.named_parameters())
        entity_property_optimizer_grouped_parameters = [{"params": [p for n, p in entity_property_param_optimizer]}]

    entity_property_optimizer = AdamW(
        entity_property_optimizer_grouped_parameters,
        lr=learning_rate,
        eps=eps
    )
    epochs = num_train_epochs
    max_grad_norm = 1.0
    total_steps = epochs * len(entity_property_train_dataloader)
    print("total_steps : ", total_steps)

    entity_property_scheduler = get_linear_schedule_with_warmup(
        entity_property_optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # polarity_model_optimizer_setting
    if FULL_FINETUNING:
        polarity_param_optimizer = list(polarity_model.named_parameters())
        no_decay = ['bias', 'gamma', 'beta']
        polarity_optimizer_grouped_parameters = [
            {'params': [p for n, p in polarity_param_optimizer if not any(nd in n for nd in no_decay)],
             'weight_decay_rate': 0.01},
            {'params': [p for n, p in polarity_param_optimizer if any(nd in n for nd in no_decay)],
             'weight_decay_rate': 0.0}
        ]
    else:
        polarity_param_optimizer = list(polarity_model.classifier.named_parameters())
        polarity_optimizer_grouped_parameters = [{"params": [p for n, p in polarity_param_optimizer]}]

    polarity_optimizer = AdamW(
        polarity_optimizer_grouped_parameters,
        lr=learning_rate,
        eps=eps
    )
    epochs = num_train_epochs
    max_grad_norm = 1.0
    total_steps = epochs * len(polarity_train_dataloader)

    polarity_scheduler = get_linear_schedule_with_warmup(
        polarity_optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )


    epoch_step = 0
    print("학습을 시작합니다...")
    for _ in trange(epochs, desc="Epoch"):
        entity_property_model.train()
        epoch_step += 1

        # entity_property train
        entity_property_total_loss = 0

        for step, batch in enumerate(entity_property_train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels, b_category_seq = batch
            # print("b_input_ids:", b_input_ids)
            # print("b_input_mask:", b_input_mask)
            # print("b_labels:", b_labels)
            # print("")

            entity_property_model.zero_grad()

            loss, _ = entity_property_model(b_input_ids, b_input_mask, b_labels, b_category_seq)

            loss.backward()

            entity_property_total_loss += loss.item()
            # print('batch_loss: ', loss.item())

            torch.nn.utils.clip_grad_norm_(parameters=entity_property_model.parameters(), max_norm=max_grad_norm)
            entity_property_optimizer.step()
            entity_property_scheduler.step()

        avg_train_loss = entity_property_total_loss / len(entity_property_train_dataloader)
        print("Entity_Property_Epoch: ", epoch_step)
        print("Average train loss: {}".format(avg_train_loss))

        model_saved_path = category_extraction_model_path + 'saved_model_epoch_' + str(epoch_step) + '.pt'
        torch.save(entity_property_model.state_dict(), model_saved_path)

        if do_eval:
            entity_property_model.eval()

            pred_list = []
            label_list = []

            for batch in entity_property_dev_dataloader:
                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels, b_category_seq = batch

                with torch.no_grad():
                    loss, logits = entity_property_model(b_input_ids, b_input_mask, b_labels, b_category_seq)

                predictions = torch.argmax(logits, dim=-1)
                pred_list.extend(predictions)
                label_list.extend(b_labels)

            evaluation(label_list, pred_list, len(tf_id_to_name))


        # polarity train
        polarity_total_loss = 0
        polarity_model.train()

        for step, batch in enumerate(polarity_train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            polarity_model.zero_grad()

            loss, _ = polarity_model(b_input_ids, b_input_mask, b_labels)

            loss.backward()

            polarity_total_loss += loss.item()
            # print('batch_loss: ', loss.item())

            torch.nn.utils.clip_grad_norm_(parameters=polarity_model.parameters(), max_norm=max_grad_norm)
            polarity_optimizer.step()
            polarity_scheduler.step()

        avg_train_loss = polarity_total_loss / len(polarity_train_dataloader)
        print("Entity_Property_Epoch: ", epoch_step)
        print("Average train loss: {}".format(avg_train_loss))

        model_saved_path = polarity_classification_model_path + 'saved_model_epoch_' + str(epoch_step) + '.pt'
        torch.save(polarity_model.state_dict(), model_saved_path)

        if do_eval:
            polarity_model.eval()

            pred_list = []
            label_list = []

            for batch in polarity_dev_dataloader:
                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels = batch

                with torch.no_grad():
                    loss, logits = polarity_model(b_input_ids, b_input_mask, b_labels)

                predictions = torch.argmax(logits, dim=-1)
                pred_list.extend(predictions)
                label_list.extend(b_labels)

            evaluation(label_list, pred_list, len(polarity_id_to_name))

    print("training is done")

In [26]:
train_sentiment_analysis()

train_sentiment_analysis
category_extraction model would be saved at  ./saved_model/category_extraction_emb/
polarity model would be saved at  ./saved_model/polarity_classification_emb/
loading train data...
tokenizing train data...
We have added 8 tokens
making dataset...
making dataloader...
loading model...


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discr

end loading
total_steps :  187580
학습을 시작합니다...


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Entity_Property_Epoch:  1
Average train loss: 0.14129454995150015
[3003, 66847]
[1536, 66147]
[0.5114885114885115, 0.9895283258784987]
accuracy:  0.9689763779527559
macro_accuracy:  0.5003389457890034
f1_score:  [0.58637144 0.9838838 ]
f1_score_micro:  0.9689763779527559
f1_score_macro:  0.7851276238352898
Entity_Property_Epoch:  1
Average train loss: 0.3314095962001011


...

[2921, 28, 54]
[2842, 15, 18]
[0.9729544676480657, 0.5357142857142857, 0.3333333333333333]
accuracy:  0.9573759573759574
macro_accuracy:  0.6140006955652282
f1_score:  [0.98033805 0.58823529 0.22929936]
f1_score_micro:  0.9573759573759574
f1_score_macro:  0.5992909015925312
Entity_Property_Epoch:  19
Average train loss: 0.012146625705917173
[3003, 66847]
[1991, 66069]
[0.663003663003663, 0.9883614821906742]
accuracy:  0.9743736578382247
macro_accuracy:  0.5504550483981124
f1_score:  [0.68988219 0.98663461]
f1_score_micro:  0.9743736578382246
f1_score_macro:  0.8382583997615956
Entity_Property_Epoch:  19
Average train loss: 0.030484878067072715


Epoch:  95%|█████████▌| 19/20 [6:47:37<21:26, 1286.71s/it]

[2921, 28, 54]
[2856, 15, 17]
[0.9777473467990414, 0.5357142857142857, 0.3148148148148148]
accuracy:  0.9617049617049617
macro_accuracy:  0.609425482442714
f1_score:  [0.98245614 0.58823529 0.24113475]
f1_score_micro:  0.9617049617049617
f1_score_macro:  0.6039420620805247
Entity_Property_Epoch:  20
Average train loss: 0.011062437473787084
[3003, 66847]
[2021, 66015]
[0.672993672993673, 0.9875536673298727]
accuracy:  0.9740300644237652
macro_accuracy:  0.5535157801078485
f1_score:  [0.69023224 0.98644691]
f1_score_micro:  0.9740300644237652
f1_score_macro:  0.838339574389106
Entity_Property_Epoch:  20
Average train loss: 0.033512031805148584


Epoch: 100%|██████████| 20/20 [7:09:03<00:00, 1287.15s/it]

[2921, 28, 54]
[2863, 15, 16]
[0.9801437863745293, 0.5357142857142857, 0.2962962962962963]
accuracy:  0.9637029637029637
macro_accuracy:  0.6040514561283704
f1_score:  [0.98351082 0.6        0.23880597]
f1_score_micro:  0.9637029637029637
f1_score_macro:  0.6074389303909856
training is done


## 모델 평가

In [30]:
def predict_from_korean_form(tokenizer, ce_model, pc_model, data):

    ce_model.to(device)
    ce_model.eval()
    for sentence in data:
        form = sentence['sentence_form']
        sentence['annotation'] = []
        if type(form) != str:
            print("form type is arong: ", form)
            continue
        for pair in entity_property_pair:
            
            tokenized_data = tokenizer(form, pair, padding='max_length', max_length=256, truncation=True)

            input_ids = torch.tensor([tokenized_data['input_ids']]).to(device)
            attention_mask = torch.tensor([tokenized_data['attention_mask']]).to(device)
            category_seq = [ i for i in range(len(entity_property_pair))] 
            category_seq = torch.tensor([category_seq]).to(device)

            with torch.no_grad():
                _, ce_logits = ce_model(input_ids, attention_mask, category_label_seq_tensor=category_seq)

            ce_predictions = torch.argmax(ce_logits, dim = -1)

            ce_result = tf_id_to_name[ce_predictions[0]]

            if ce_result == 'True':
                with torch.no_grad():
                    _, pc_logits = pc_model(input_ids, attention_mask)

                pc_predictions = torch.argmax(pc_logits, dim=-1)
                pc_result = polarity_id_to_name[pc_predictions[0]]

                sentence['annotation'].append([pair, pc_result])


    return data

## F1 score 계산 - 추출 성능 및 전체 성능에 대한 F1 score 따로 계산

In [31]:
def evaluation_f1(true_data, pred_data):

    true_data_list = true_data
    pred_data_list = pred_data

    ce_eval = {
        'TP': 0,
        'FP': 0,
        'FN': 0,
        'TN': 0
    }

    pipeline_eval = {
        'TP': 0,
        'FP': 0,
        'FN': 0,
        'TN': 0
    }

    for i in range(len(true_data_list)):

        # TP, FN checking
        is_ce_found = False
        is_pipeline_found = False
        for y_ano  in true_data_list[i]['annotation']:
            y_category = y_ano[0]
            y_polarity = y_ano[2]

            for p_ano in pred_data_list[i]['annotation']:
                p_category = p_ano[0]
                p_polarity = p_ano[1]

                if y_category == p_category:
                    is_ce_found = True
                    if y_polarity == p_polarity:
                        is_pipeline_found = True

                    break

            if is_ce_found is True:
                ce_eval['TP'] += 1
            else:
                ce_eval['FN'] += 1

            if is_pipeline_found is True:
                pipeline_eval['TP'] += 1
            else:
                pipeline_eval['FN'] += 1

            is_ce_found = False
            is_pipeline_found = False

        # FP checking
        for p_ano in pred_data_list[i]['annotation']:
            p_category = p_ano[0]
            p_polarity = p_ano[1]

            for y_ano  in true_data_list[i]['annotation']:
                y_category = y_ano[0]
                y_polarity = y_ano[2]

                if y_category == p_category:
                    is_ce_found = True
                    if y_polarity == p_polarity:
                        is_pipeline_found = True

                    break

            if is_ce_found is False:
                ce_eval['FP'] += 1

            if is_pipeline_found is False:
                pipeline_eval['FP'] += 1

    ce_precision = ce_eval['TP']/(ce_eval['TP']+ce_eval['FP'])
    ce_recall = ce_eval['TP']/(ce_eval['TP']+ce_eval['FN'])

    ce_result = {
        'Precision': ce_precision,
        'Recall': ce_recall,
        'F1': 2*ce_recall*ce_precision/(ce_recall+ce_precision)
    }

    pipeline_precision = pipeline_eval['TP']/(pipeline_eval['TP']+pipeline_eval['FP'])
    pipeline_recall = pipeline_eval['TP']/(pipeline_eval['TP']+pipeline_eval['FN'])

    pipeline_result = {
        'Precision': pipeline_precision,
        'Recall': pipeline_recall,
        'F1': 2*pipeline_recall*pipeline_precision/(pipeline_recall+pipeline_precision)
    }

    return {
        'category extraction result': ce_result,
        'entire pipeline result': pipeline_result
    }

## 테스트 데이터에 대한 평가

In [37]:
def test_sentiment_analysis():

    tokenizer = ElectraTokenizer.from_pretrained(base_model)
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
    test_data = jsonlload(dev_data_path)

    entity_property_test_data, polarity_test_data = get_dataset(test_data, tokenizer, max_len)

    entity_property_test_dataloader = DataLoader(entity_property_test_data, shuffle=True,
                                batch_size=batch_size)

    polarity_test_dataloader = DataLoader(polarity_test_data, shuffle=True,
                                                  batch_size=batch_size)
    
    model =  CategoryClassification(len(tf_id_to_name), len(tokenizer), lstm_hidden * 2, len(entity_property_pair), lstm_num_layer, bilstm_flag)
    model.load_state_dict(torch.load(test_category_extraction_model_path, map_location=device))
    model.to(device)
    model.eval()
            
    polarity_model = RoBertaBaseClassifier(len(polarity_id_to_name), len(tokenizer))
    polarity_model.load_state_dict(torch.load(test_polarity_classification_model_path, map_location=device))
    polarity_model.to(device)
    polarity_model.eval()

    pred_data = predict_from_korean_form(tokenizer, model, polarity_model, copy.deepcopy(test_data))

    # jsondump(pred_data, './pred_data.json')
    # pred_data = jsonload('./pred_data.json')

    print('F1 result: ', evaluation_f1(test_data, pred_data))

In [38]:
test_sentiment_analysis()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discr

F1 result:  {'category extraction result': {'Precision': 0.7505446623093682, 'Recall': 0.6713218577460215, 'F1': 0.708726212926453}, 'entire pipeline result': {'Precision': 0.7357531760435572, 'Recall': 0.6583306268268918, 'F1': 0.6948920123414466}}
